In [1]:
# Veja mais informações no post do Medium

# https://medium.com/@conversandoemcodigos/resolvendo-o-letreco-de-maneira-aut%C3%B4noma-76373fcfbd36
#Importando as bibliotecas. 

#Importando as bibliotecas. 
# Para usar o Selenium, é necessário baixar também o drive compatível

# Biblioteca para Automação Web
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# biblioteca para genrenciar banco de dados
import pandas as pd

# Biblioteca pra gerar números aleatórios
from random import randint


from bs4 import BeautifulSoup 
from lxml import html

In [2]:
# Essa função define a cor da letra e coloca dentro de uma lista com a respectiva cor.
# A variável x é a posição e a letra é o elemento encontrado
def cor_letra(x, letra):
    
    # só faz a busca em posições vazias da lista
    if len(verde[x]) == 0:
        if letra == 'letter-wrapper rounded d-flex justify-content-center align-items-center right me-2' or letra == 'letter-wrapper rounded d-flex justify-content-center align-items-center right':
            verde[x] = palavra_digitada[x]
        elif letra == "letter-wrapper rounded d-flex justify-content-center align-items-center displaced" or letra== 'letter-wrapper rounded d-flex justify-content-center align-items-center displaced me-2':
            amarelo[x] = palavra_digitada[x]         
        elif letra == "letter-wrapper rounded d-flex justify-content-center align-items-center wrong" or letra== "letter-wrapper rounded d-flex justify-content-center align-items-center wrong me-2":
            vermelho[x] = palavra_digitada[x]

        
# Essa função define em qual linha do site vai ser trabalhado e depois usa função anterior
# para definir a cor da letra. A linha é o número da fileira do Letreco e vai de 1 a 5      

def fila(linha):
    letra0 = navegador.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[1]').get_attribute('class')
    letra1 = navegador.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[2]').get_attribute('class')
    letra2 = navegador.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[3]').get_attribute('class')
    letra3 = navegador.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[4]').get_attribute('class')
    letra4 = navegador.find_element(By.XPATH, f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[5]').get_attribute('class')
    cor_letra(0, letra0)
    cor_letra(1, letra1)
    cor_letra(2, letra2)
    cor_letra(3, letra3)
    cor_letra(4, letra4)

    
# Função para usar o teclado do site. 
# palavra é a palavra que será digitada e site é o html do Letreco

def digitar_palavra(palavra, site):
    
    # transforma o html do site em string
    doc = html.fromstring(site)
    
    # cada uma das letras é buscada no teclado do Letreco
    for letra in palavra:
        
        # busca o elemento 
        element = doc.xpath(f'//span[contains(text(),"{letra.upper()}")]')[-1]

        # Obtem o XPath do elemento
        xpath = element.getroottree().getpath(element)
        
        # clica no xpath gerado
        navegador.find_element(By.XPATH, xpath).click()




In [9]:
palavras = pd.read_csv('termo.csv')
palavras.sample(5)

,palavras
5410,temor
3655,solfo
4317,hotel
6054,nimbo
487,muita


In [12]:
# Solicitando a palavra
palavra_digitada = input('Digite uma palavra de cinco letras: ')

# Importando as palavras e colocando cada letra em uma coluna que marca sua posição
palavras = pd.read_csv('termo.csv')
palavras['lista'] = palavras['palavras'].apply(list)
palavras[[0, 1, 2, 3, 4]] = pd.DataFrame(palavras['lista'].apply(pd.Series))

# Imprimindo o número total de palavras no dataset
print(f'- O número total de palavras é: {len(palavras)}')

# Estabelecendo as listas
verde = [[],[],[],[],[]]
vermelho = [[],[],[],[],[]]
amarelo = [[],[],[],[],[]]


# Entrando no site, maximilizando para poder usar o telcado e digitando a palavra escolhida

navegador = webdriver.Chrome()
navegador.get('https://www.gabtoschi.com/letreco/')
navegador.maximize_window()
site = navegador.page_source
digitar_palavra(palavra_digitada, site)
navegador.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/button[2]').click()

# Mudando de fila automaticamente
for num in range(1, 6):
    try:
        fila(num)
        for c in range(0, 5):
            

            # Verde: elimina todas as palavras que não são com a letra determinada na posição 

            if len(verde[c]) == 1:
                
                palavras = palavras[palavras[c] == verde[c][0]]
                
                

            # Amarelo: primeiro elimina quem não tem a letra amarela e depois apaga
            # quem tem a letra naquela posição

            elif len(amarelo[c]) == 1:
                palavras = palavras[palavras['palavras'].str.contains(amarelo[c][0])]
                palavras = palavras[palavras[c] != amarelo[c][0]]
                

            # Vermelho tem três casos

            elif len(vermelho[c]) == 1:

                # quando a letra vermelha não se repete nas letras amarelas e nas letras verdes,
                # todas as palavras com essa letra são eliminadas
                if vermelho[c] not in amarelo and vermelho[c][0] not in verde:

                    palavras = palavras[~palavras['palavras'].str.contains(vermelho[c][0])]

                # quando a letra vermelha aparece também na amarela, apenas são excluídos
                # quem não tem a letra naquela coluna
                elif vermelho[c] in amarelo:
                    palavras = palavras[palavras[c] != vermelho[c][0]]

                
                #quando a letra vermelha aparece também com a verde, são excluídas todas as colunas que tem aquela letra, 
                # menos a que tem a verde 
                elif vermelho[c] in verde:
                    for pos_verde in range(0, 5):
                        if vermelho[c] != verde[pos_verde]:
                            palavras = palavras[palavras[pos_verde] != vermelho[c]]




        # Limpeza das fileiras 
        vermelho = [[],[],[],[],[]]
        amarelo = [[],[],[],[],[]]

        # Resetando o índice e gerando uma nova palavra para ser digitada 
        palavras.reset_index(drop=True, inplace=True)
        print(f'O número total de palavras é {len(palavras)}')
        
        # escolhe uma nova palavra
        palavra_digitada = list(palavras.sample()['palavras'])[0]
        digitar_palavra(palavra_digitada, site)
        
        navegador.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/button[2]').click()
        
    # Quando a palavra é encontrada em menos de seis tentativas se encerra e mostra o número de palavras que sobraram
    except:
        
        break
        
print(f'- O total de palavras é: {len(palavras)}')



Digite uma palavra de cinco letras: nimbo
- O número total de palavras é: 10205
O número total de palavras é 1072
O número total de palavras é 48
O número total de palavras é 2
O número total de palavras é 1
O número total de palavras é 1
- O total de palavras é: 1


In [11]:
# apaga a palavra 

apagar = input('Qual palavra quer apagar: ')
palavras = pd.read_csv('termo.csv')
palavra_index = palavras.query(f'palavras == "{apagar}"').index[0]
palavras = palavras.drop(palavra_index)
palavras.to_csv('termo.csv', index=False)

Qual palavra quer apagar: reias
